In [339]:
import pandas as pd
import json
import numpy as np
import tqdm

### smes data

In [85]:
with open('./data/companies_info.json') as f:
    js = json.loads(f.read())

In [111]:
smes_company = pd.DataFrame(js).transpose()
smes_company = smes_company.reset_index().rename(columns={'index':'회사명'})

In [134]:
smes_company_22 = smes_company[smes_company['latest_year'] == '2022']
smes_company_22.columns = ['회사명', '업종명', '주생산품', 'year', '부채총액', '현금', '자본총계', '매출액', '영업이익(EBIT)', '감가상각비', '당기순이익']
smes_company_22 = smes_company_22.reset_index()
smes_company_22 = smes_company_22.drop(['index'], axis=1)

In [160]:
idx_after = ['부채총액', '현금', '자본총계', '매출액', '영업이익(EBIT)', '감가상각비', '당기순이익']

for j, idx in enumerate(smes_company_22.columns[4:11]):
    temp = smes_company_22[idx].apply(pd.Series)
    smes_company_22[idx_after[j]] = temp[0].str.replace(',', '').apply(lambda x: int(x))

del_idx_set = set()
for j in idx_after:
    if j == '감가상각비': continue
    del_idx_set.update(smes_company_22[smes_company_22[j]==0]['회사명'].index)

In [ ]:
smes_company_22 = smes_company_22.drop(del_idx_set).reset_index(drop=True)

smes_company_22['순부채'] = smes_company_22['부채총액'] - smes_company_22['현금']
smes_company_22['상각전영업이익(EBITDA)'] = smes_company_22['영업이익(EBIT)'] + smes_company_22['감가상각비']
smes_company_22 = smes_company_22.drop(['liab', 'cash', 'equi', 'reve', 'oper', 'depr', 'neti'], axis=1)
smes_company_22['상장분류'] = False

### dart data

In [ ]:
dart_company = pd.read_csv('./data/final_df.csv')
cols = ['부채총계', '현금및현금성자산', '자본총계', '매출액', '영업이익', '감가상각비', '당기순이익']

Null 값 & 중복 값 제거

In [408]:
del_idx_set = set()
for j in cols:
    if j == '감가상각비': continue
    del_idx_set.update(dart_company[dart_company[j].isna() == True].index)
    
dart_company = dart_company.drop(del_idx_set).reset_index(drop=True)
dart_company = dart_company.drop_duplicates()

2022년 사업보고서만 추출

In [444]:
dart_company_22 = dart_company[(dart_company['report_code'] == 11011) & (dart_company['year'] == 2022)]

column 통일을 위한 작업

In [446]:
dart_company_22 = dart_company_22.drop(['유동자산', '유동부채', '자본금', '비유동자산', '비유동부채', '이익잉여금', '법인세차감전 순이익', 'year', 'report_code', '구분', '발행할 주식의 총수', '현재까지 발행한 주식의 총수',
       '현재까지 감소한 주식의 총수', '현재까지 감소한 주식의 총수(감자)', '현재까지 감소한 주식의 총수(이익소각)',
       '현재까지 감소한 주식의 총수(상환주식의 상환)', '현재까지 감소한 주식의 총수(기타)', '자기주식수',
       '유통주식수'], axis = 1)
dart_company_22.columns = ['회사명', '자산총계', '부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '법인유형',
       '화폐', '결산 월', '발행주식의 총수', '업종', '업종명', '현금', '감가상각비']
dart_company_22.loc[dart_company_22['감가상각비'].isna(), '감가상각비'] = 0
dart_company_22['순부채'] = dart_company_22['부채총액'] - dart_company_22['현금']
dart_company_22['상각전영업이익(EBITDA)'] = dart_company_22['영업이익(EBIT)'] + dart_company_22['감가상각비']
dart_company_22['상장분류'] = True

KRX 크롤링으로 주가 column 채우기

In [492]:
corp = pd.read_csv('./data/corp.csv')

- 주식코드 변경으로 인해 트래킹이 되지 않았음. 수작업처리

In [ ]:
corp_stock = corp[corp['stock_code']!=' '][['corp_name', 'stock_code']]
corp_stock.columns=['회사명', '종목코드']

In [ ]:
dup_codes = corp_stock.groupby(['회사명']).filter(lambda x : len(x) > 1)['회사명'].unique()
for dc in dup_codes:
    if dc == '원텍' or dc == '원익피앤이':
        # 위에 있는 값 사용
        corp_stock = corp_stock.drop_duplicates(['회사명'], keep='first')
    else:
        # 아래에 있는 값 사용
        corp_stock = corp_stock.drop_duplicates(['회사명'], keep='last')

In [ ]:
for dc in dup_codes:
    if dc == '두산건설':
        corp = corp.drop(index = corp[corp['corp_name'] == dc].index, axis=0)
    if dc == '국민은행':
        corp = pd.concat([corp, pd.DataFrame({'corp_code':[np.NaN], 'corp_name': [dc], 'stock_code': ['105560'], 'modify_date':['20230712']})], ignore_index=True)
    if dc == '하나은행':
        corp = pd.concat([corp, pd.DataFrame({'corp_code':[np.NaN], 'corp_name': [dc], 'stock_code': ['086790'], 'modify_date':['20230712']})], ignore_index=True)

corp & dart data merge

In [ ]:
dart_company_22 = pd.merge(dart_company_22, corp_stock, on='회사명', how='left')
dart_company_22['주가'] = np.NaN

In [ ]:
from pykrx import stock
error_idx = set()
last_day = ['31', '28', '31', '30', '31', '30', '31', '31', '30', '31', '30', '31']

for i, data in enumerate(dart_company_22['종목코드']):
    try:
        month = int(dart_company_22['결산 월'][i])
        s_month = str(month).zfill(2)
        stocks = stock.get_market_ohlcv(f'2022{s_month}01', f'2022{s_month}{last_day[month-1]}', data, 'm')
        dart_company_22.loc[i, '주가'] = stocks['종가'][0]
    except (KeyError, IndexError):
        error_idx.add(i)
dart_company_22 = dart_company_22.drop(index=dart_company_22[dart_company_22['주가'].isna()].index, axis = 0).reset_index(drop=True)
dart_company_22 = dart_company_22.rename(columns={'업종':'업종코드'})
dart_company_22.to_csv('./data/dart_comp_22.csv', index=False)

### All data

In [ ]:
all_company = pd.DataFrame(columns = ['회사명', '업종명', '상장분류', '주가', '발행주식의 총수', '시가총액', '부채총액', '현금', '순부채', '기업총가치', '자본총계', '매출액', '영업이익(EBIT)', '감가상각비', '상각전영업이익(EBITDA)', '당기순이익', 'EBITDA배율', 'EBIT배율', 'PER배율', 'PBR배율', 'PSR배율', '기업 가치'])
all_company_22 = pd.concat([all_company, smes_company_22])
all_company_22 = pd.concat([all_company_22, dart_company_22])

In [ ]:
all_company_22['시가총액'] = all_company_22['주가'] * all_company_22['발행주식의 총수']
all_company_22['시가총액'] = all_company_22['시가총액'].replace(np.nan, 0)
all_company_22['기업총가치'] = all_company_22['시가총액'] + all_company_22['순부채']

- EBITDA 배율

In [ ]:
all_company_22 = all_company_22.drop(index = list(all_company_22[all_company_22['상각전영업이익(EBITDA)'] == 0]['영업이익(EBIT)'].index), axis=0).reset_index(drop=True)
all_company_22['EBITDA배율'] = all_company_22['기업총가치'] / all_company_22['상각전영업이익(EBITDA)']

- EBIT 배율

In [ ]:
all_company_22 = all_company_22.drop(index = list(all_company_22[all_company_22['영업이익(EBIT)'] == 0]['영업이익(EBIT)'].index), axis=0).reset_index(drop=True)
all_company_22['EBIT배율'] = all_company_22['기업총가치'] / all_company_22['영업이익(EBIT)']

- PER 배율

In [ ]:
all_company_22 = all_company_22.drop(index = list(all_company_22[all_company_22['당기순이익'] == 0]['당기순이익'].index), axis=0).reset_index(drop=True)
all_company_22['PER배율'] = all_company_22['시가총액'] / all_company_22['당기순이익']

- PBR 배율

In [ ]:
all_company_22 = all_company_22.drop(index = list(all_company_22[all_company_22['자본총계'] == 0]['자본총계'].index), axis=0).reset_index(drop=True)
all_company_22['PBR배율'] = all_company_22['시가총액'] / all_company_22['자본총계']

- PSR 배율

In [ ]:
all_company_22 = all_company_22.drop(index = list(all_company_22[all_company_22['매출액'] == 0]['매출액'].index), axis=0).reset_index(drop=True)
all_company_22['PSR배율'] = all_company_22['기업총가치'] / all_company_22['매출액']

- 필요없는 column drop

In [ ]:
all_company_22 = all_company_22.drop(['year', '결산 월', '업종코드', '법인유형', '자산총계', '기업 가치'], axis=1)
all_company_22 = all_company_22.drop(index = all_company_22[all_company_22['화폐'] == 'USD'].index, axis=0)
all_company_22 = all_company_22.drop(index = all_company_22[all_company_22['화폐'] == 'JPY'].index, axis=0)
all_company_22 = all_company_22.drop(['화폐'], axis=1)
all_company_22.to_csv('./data/all_data_2022.csv', index=False)